### Roles Identification

In [1]:
import pandas as pd
import networkx as nx

#Exercise #1, #2, #3, #4
for k in range(1,5):
    #Data input
    df = pd.read_excel('CE_4220.xlsx', 'Exercise' + str(k))
    
    #Collaborative learning network
    DG = nx.DiGraph()
    DG.add_nodes_from(df.iloc[:,0])
    for j in range(1, df.shape[1]):
        for index,value in enumerate(df.iloc[:,j]):
            if not pd.isna(value):
                DG.add_edge(df.iloc[:,0][index], j, weight=value)
    
    #Centrality measures
    centrality = pd.DataFrame({'node': df.iloc[:,0].tolist(),
                               'in_degree': dict(DG.in_degree(weight='weight')).values(),
                               'out_degree': dict(DG.out_degree(weight='weight')).values(),
                               'betweenness': nx.betweenness_centrality(DG, weight='weight').values(),
                               'closeness': nx.closeness_centrality(DG).values()})
    
    #Subgraph labeling
    subgraph = list(nx.weakly_connected_components(DG))
    for i in range(len(subgraph)):
        for node in subgraph[i]:
            centrality.loc[centrality[centrality['node'] == node].index.tolist()[0], 'label'] = i + 1
    
    #Group by subgraphs
    group = centrality.groupby('label', as_index=False).mean()
    
    #Centrality labeling criterion
    def centrality_labeling(x, v):
        return 2 if x >= v else 1 if x > 0 else 0
    
    #Categorical centrality table
    centrality_c = centrality[['node','in_degree','out_degree','betweenness','closeness']]
    for col in centrality.iloc[:,1:5].columns:
        for row in range(centrality.shape[0]):
            v = group.loc[group[group['label'] == centrality.loc[row,'label']].index.tolist()[0],col]
            centrality_c.loc[row,col] = centrality_labeling(centrality.loc[row,col], v)
    
    #Roles: Leader, Animator, Active, Peripheral, Quiet, Missing
    #Roles identification criterion
    def role_identification(in_degree, out_degree, betweenness, closeness):
        if out_degree == 2:
            if betweenness == 2:
                if in_degree == 2 and closeness == 2:
                    return 'Leader'
                else:
                    return 'Animator'
            else:
                return 'Active'
        elif out_degree == 1:
            return 'Peripheral'
        else:
            if in_degree == 0 and closeness == 0:
                return 'Missing'
            else:
                return 'Quiet'
    
    for row in range(centrality_c.shape[0]):
        parameters = tuple(centrality_c.loc[row,['in_degree','out_degree','betweenness','closeness']])
        centrality_c.loc[row,'role'] = role_identification(*parameters)
    
    #Data output
    pd.concat([centrality['node'], centrality_c['role']], axis=1).to_csv('Exercise_' + str(k) + '.csv', index=False)

### Agent Performance

In [3]:
import pandas as pd

role_score = {'Leader':4, 'Animator':3, 'Active':2, 'Peripheral':1, 'Quiet':0, 'Missing':0}

#Agent roles
agent_role = pd.DataFrame({'agent': centrality['node']})
for k in range(1,5):
    agent_role['Exercise_' + str(k)] = pd.read_csv('Exercise_' + str(k) + '.csv')['role']

#Overall score for each agent
agent_score = agent_role
agent_score.iloc[:,1:] = agent_score.iloc[:,1:].applymap(lambda x: role_score[x])
agent_score['overall_score'] = agent_score.iloc[:,1:].apply(lambda x: x.mean(), axis=1)

#Data output
#agent_score.to_csv('Agent_score.csv', index=False)
agent_score.head()

,agent,Exercise_1,Exercise_2,Exercise_3,Exercise_4,overall_score
0,1,4,1,4,1,2.50
1,2,3,3,1,1,2.00
2,3,1,0,1,1,0.75
3,4,2,3,2,1,2.00
4,5,3,3,4,4,3.50


In [4]:
import numpy as np
import pandas as pd

roles = ['Leader','Animator','Active','Peripheral','Quiet','Missing']
agent_role = pd.DataFrame({'agent': centrality['node']})
exercise_stat = pd.DataFrame()

for k in range(1,5):
    #Agent roles
    agent_role['Exercise_' + str(k)] = pd.read_csv('Exercise_' + str(k) + '.csv')['role']
    
    #Exercise statistics
    exercise_stat.loc[k-1, 'Exercise'] = 'Exercise_' + str(k)
    for item in roles:
        if item in agent_role['Exercise_' + str(k)].values:
            exercise_stat.loc[k-1, item] = agent_role['Exercise_' + str(k)].value_counts()[item]

#Data type conversion
exercise_stat = exercise_stat.replace(np.nan, 0)
exercise_stat.iloc[:,1:] = exercise_stat.iloc[:,1:].astype(int)

#Data output
#exercise_stat.to_csv('Exercise_stat.csv', index=False)
exercise_stat.head()

,Exercise,Leader,Animator,Active,Peripheral,Quiet
0,Exercise_1,1,10,2,7,9
1,Exercise_2,1,9,1,9,9
2,Exercise_3,6,10,3,7,3
3,Exercise_4,2,8,6,11,2
